# 2 Namespaces and Functions 

### 2.1 Functions

The definition of a function adds its name to the namespace.

In [ ]:
def add(a, b):
    return a + b

In [ ]:
add

In [ ]:
add.__qualname__

In [ ]:
add.__qualname__ = 'multiply'

In [ ]:
multiply

In [ ]:
add.__qualname__

In [ ]:
add

In [ ]:
multiply = add

In [ ]:
multiply

In [ ]:
add.__dict__

In [ ]:
add.spam = 'eggs'
add.__dict__

  Function execution defines a new scope.  When the function is
called, a namespace is created and the parameter names are bound to
the function call's arguments.

In [ ]:
def add(a, b):
    return a + b
add(2, 3)

  Note that default argument values are expressions evaluated when the function is defined.

In [ ]:
def f(p1=print('When are default args evaluated?')):
    print(f'p1 has value {p1}')
    pass

In [ ]:
f()

In [ ]:
def f1(p1=print('p1 defined')):
    def f2(p2=print('p2 defined')):
        pass
    f2()
    return 'from f1'

In [ ]:
f1()

In [ ]:
f1()

In [ ]:
f1.__code__

In [ ]:
f1.__code__.co_consts

In [ ]:
f1.__code__.co_consts[2]

In [ ]:
import dis
dis.dis(f1.__code__.co_consts[2])

In [ ]:
dis.dis(f1.__code__)

  Function annotations are arbitrary expressions.


In [ ]:
def add(a: int, b: int, verbose: bool) -> int:
    return a + b

In [ ]:
add.__annotations__

In [ ]:
type(add.__annotations__)

In [ ]:
def add(a: len('abcd'), b: 99 + 1, verbose: object()) -> 'A string literal':
    return a + b

In [ ]:
add.__annotations__

### 2.2 Exercises: Functions

In [1]:
def f(*args, **kwargs):
    print(f'{args!r}\n{kwargs!r}')

In [2]:
f(1)

(1,)
{}


In [3]:
f(1, 2)

(1, 2)
{}


In [4]:
f(1, a=3, b=4)

(1,)
{'a': 3, 'b': 4}


In [5]:
t = 1, 2
t

(1, 2)

In [6]:
d = dict(a=3, b=4)
d

{'a': 3, 'b': 4}

In [7]:
list(d)

['a', 'b']

In [8]:
f(t)

((1, 2),)
{}


In [9]:
f(d)

({'a': 3, 'b': 4},)
{}


In [10]:
f(*t)

(1, 2)
{}


In [11]:
f(*d)

('a', 'b')
{}


In [12]:
f(**t)

TypeError: f() argument after ** must be a mapping, not tuple

In [13]:
f(**d)

()
{'a': 3, 'b': 4}


### 2.3 Scopes and Search Order

Name binding operations covered so far:

  - *name* `=` (assignment)
  - `del` *name* (unbinds the name)
  - `def` *name* function definition (including lambdas)
  - `def name(`*names*`):` (function execution)
  - *name*`.`*attribute_name* `=`, `__setattr__`, `__delattr__`


Review:
- A *namespace* is a mapping from names to objects.
  Think of it as a dictionary.

- Simple assignment (`a_name =`) and simple `del` (`del
  an_identifier`) of a name are namespace operations, not operations
  on objects.

Terminology and Definitions:
- A *scope* is a section of Python code where a namespace is *directly*
  accessible, by using a *name*.

- For an *indirectly* accessible namespace you access values via dot
  notation, e.g. `math.pi` or `sys.version_info.major`.

- The (*direct*) namespace search order is (from http://docs.python.org/3/tutorial):

  - The innermost scope contains local names

  - The namespaces of enclosing functions, searched starting
    with the nearest enclosing scope; (or the module if outside any
    function)

  - The middle scope contains the current module's global names

  - The outermost scope is the namespace containing built-in
    names

- All namespace *changes* happen in the local scope (i.e. in the
  current scope in which the namespace-changing code executes).

  You should avoid reassigning built-in names because it may mislead
  the reader, but let's do so to explore how name scopes work:


In [ ]:
len

In [ ]:
def f():
    print('Line A', len)

In [ ]:
f()

In [ ]:
def f():
    def len():
        pass
    print(len)

In [ ]:
f()

In [ ]:
def f():
    def len():
        print('Line B', len)
        pass
    len()
    print('Line C', len)

In [ ]:
f()

In [ ]:
def f():
    def len():
        len = 'short'
        print('Line E', len)
    print('Line F', len)
    len()

In [ ]:
f()

In [ ]:
len

In [ ]:
len = 99

In [ ]:
len

In [ ]:
def print_len(s):
    print('len(s):', len(s))

In [ ]:
print_len('walk')

In [ ]:
len

In [ ]:
del len

In [ ]:
len

In [ ]:
del len

In [ ]:
len

In [ ]:
print_len('walk')

In [ ]:
pass

In [ ]:
pass = 3

In [ ]:
import keyword
print(' '.join(keyword.kwlist))

  Keywords are the only identifiers (not names) that don't refer to
objects, and they don't really exist as keywords at run-time (except
inside `eval` which is compile-time).

### 2.4 Function Locals

Let's look at some surprising behaviour.

In [ ]:
value = 'module'
def test_outer_scope():
    print(value)

In [ ]:
test_outer_scope()

In [ ]:
def test_local():
    value = 'inner'
    print(value)

In [ ]:
value

In [ ]:
test_local()

In [ ]:
def test_unbound_local():
    print(value)
    value = 'inner'

In [ ]:
value

In [ ]:
test_unbound_local()

In [ ]:
value

  Let's look at the function `test_unbound_local` to help us understand this error.

In [ ]:
test_unbound_local.__code__

In [ ]:
test_unbound_local.__code__.co_argcount  # count of positional args

In [ ]:
test_unbound_local.__code__.co_name  # function name (3 places)

In [ ]:
test_unbound_local.__code__.co_names  # names used in bytecode

In [ ]:
test_unbound_local.__code__.co_nlocals  # number of locals

In [ ]:
test_unbound_local.__code__.co_varnames  # names of locals

  See "Code objects" at https://docs.python.org/3/reference/datamodel.html?highlight=co_nlocals#the-standard-type-hierarchy

In [ ]:
def test_local():
    value = 'inner'
    print(value)

In [ ]:
def test_unbound_local():
    print(value)
    value = 'inner'

In [ ]:
test_local.__code__.co_consts, test_local.__code__.co_consts

In [ ]:
print('test_local')

In [ ]:
dis.dis(test_local.__code__.co_code)

In [ ]:
print('test_unbound_local')

In [ ]:
dis.dis(test_unbound_local.__code__.co_code)

In [ ]:
import dis

In [ ]:
import io

In [ ]:
import itertools

In [ ]:
def dis_column(obj):
    str_file = io.StringIO()
    dis.dis(obj, file=str_file)
    return [line[14:] for line in str_file.getvalue().splitlines()]

In [ ]:
def columnar(col1, col2):
    width = max(len(row) for row in col1)
    [f'{s1:{width}}   {s2}' for s1, s2 in itertools.zip_longest(col1, col2)]
    print('\n'.join(f'{s1:{width}}   {s2}' for s1, s2 in itertools.zip_longest(col1, col2)))

In [ ]:
columnar(dis_column(test_local), dis_column(test_unbound_local))    

  Same bytecode, but different order - LOAD_FAST before STORE_FAST is the problem.


> "This is because when you make an assignment to a variable in a
> scope, that variable becomes local to that scope and shadows any
> similarly named variable in the outer scope. Since the last
> statement in foo assigns a new value to x, the compiler recognizes
> it as a local variable. Consequently when the earlier print x
> attempts to print the uninitialized local variable and an error
> results." --
> https://docs.python.org/3/faq/programming.html#why-am-i-getting-an-unboundlocalerror-when-the-variable-has-a-value


  To explore this further on your own compare these two:

In [ ]:
import codeop

In [ ]:
dis.dis(codeop.compile_command('def t1(): a = b; b = 7'))

In [ ]:
dis.dis(codeop.compile_command('def t2(): b = 7; a = b'))

  What about `global`?

In [ ]:
def test_global():
    # print('A -->', value)
    global value
    print('B -->', value)
    value = 'inner'  # This assignment is what makes value local
    print('C -->', value)

In [ ]:
value

In [ ]:
test_global()

In [ ]:
value

In [ ]:
test_global.__code__.co_varnames

  Note LOAD_GLOBAL instead of LOAD_FAST:

In [ ]:
dis.dis(test_global.__code__.co_code)

### 2.5 Exercises: Function Non-Locals

  Python 3 added `nonlocal`.

In [33]:
def test_nonlocal():
    x = 5
    def assign_6():
        nonlocal x
        print(2, x)
        x = 6
        print(3, x)
    print(1, x)
    assign_6()
    print(4, x)

In [34]:
x = 'module'

In [35]:
x

'module'

In [36]:
test_nonlocal()

1 5
2 5
3 6
4 6


In [18]:
x

'module'

In [19]:
def f():
    print(x)
    def x():
        pass

In [20]:
f()

UnboundLocalError: local variable 'x' referenced before assignment

In [21]:
def f2():
    del len

In [22]:
f2()

UnboundLocalError: local variable 'len' referenced before assignment

  See also https://docs.python.org/3/tutorial/classes.html#scopes-and-namespaces-example

### 2.6 Built-ins

Restart Python to unclutter the namespace.

In [ ]:
%%javascript
IPython.notebook.kernel.restart();

In [ ]:
[n for n in dir() if not n.startswith('_')]

  There are lots of built-in names that `dir()` doesn't show us.
Let's use some Python to explore all the builtin names by category.

In [ ]:
import builtins, collections, inspect, textwrap
fill = textwrap.TextWrapper(width=60).fill
def pfill(pairs):
    """Sort and print first of every pair"""
    print(fill(' '.join(list(zip(*sorted(pairs)))[0])))

  Collect all members of `builtins`:

In [ ]:
members = set([
    m for m in inspect.getmembers(builtins)
    if not m[0].startswith('_')])
len(members)

  Pull out only the `exception` types:

In [ ]:
exceptions = [
    (name, obj) for (name, obj) in members
    if inspect.isclass(obj) and
    issubclass(obj, BaseException)]
members -= set(exceptions)
len(exceptions), len(members)

In [ ]:
pfill(exceptions)

https://docs.python.org/3/library/exceptions.html#exception-hierarchy:

    BaseException
     +-- SystemExit
     +-- KeyboardInterrupt
     +-- GeneratorExit
     +-- Exception           <---- NB
          +-- StopIteration
          +-- StopAsyncIteration
          +-- ArithmeticError
          |    +-- FloatingPointError
          |    +-- OverflowError
          |    +-- ZeroDivisionError
          +-- AssertionError
          +-- AttributeError
          +-- BufferError
          +-- EOFError
          +-- ImportError
               +-- ModuleNotFoundError
          +-- LookupError
          |    +-- IndexError
          |    +-- KeyError
          +-- MemoryError
          +-- NameError
          |    +-- UnboundLocalError
          +-- OSError
          |    +-- BlockingIOError
          |    +-- ChildProcessError
          |    +-- ConnectionError
          |    |    +-- BrokenPipeError
          |    |    +-- ConnectionAbortedError
          |    |    +-- ConnectionRefusedError
          |    |    +-- ConnectionResetError
          |    +-- FileExistsError
          |    +-- FileNotFoundError
          |    +-- InterruptedError
          |    +-- IsADirectoryError
          |    +-- NotADirectoryError
          |    +-- PermissionError
          |    +-- ProcessLookupError
          |    +-- TimeoutError
          +-- ReferenceError
          +-- RuntimeError
          |    +-- NotImplementedError
          |    +-- RecursionError
          +-- SyntaxError
          |    +-- IndentationError
          |         +-- TabError
          +-- SystemError
          +-- TypeError
          +-- ValueError
          |    +-- UnicodeError
          |         +-- UnicodeDecodeError
          |         +-- UnicodeEncodeError
          |         +-- UnicodeTranslateError
          +-- Warning
               +-- DeprecationWarning
               +-- PendingDeprecationWarning
               +-- RuntimeWarning
               +-- SyntaxWarning
               +-- UserWarning
               +-- FutureWarning
               +-- ImportWarning
               +-- UnicodeWarning
               +-- BytesWarning
               +-- ResourceWarning

In [ ]:
pfill(members)

  If you want to catch all exceptions use except Exception, not bare except:

In [ ]:
try:
    1/0
except Exception as e:
    print('The exception raised was', e)
    raise

  Most are either of type `type`, or `builtin_function_or_method`:

In [ ]:
type(int), type(len)

  Print them:

In [ ]:
bnames = collections.defaultdict(set)
for name, obj in members:
    bnames[type(obj)].add((name, obj))
for typ in [type(int), type(len)]:
    pairs = bnames.pop(typ)
    print(typ)
    pfill(pairs)
    print()

  The leftovers:

In [ ]:
for typ, pairs in bnames.items():
    print('{}: {}'.format(typ, ' '.join((n for (n, o) in pairs))))